In [4]:
from math import radians, sin, cos, sqrt, asin
import time

s = time.time()

path = "/Users/mrunmayee/DistributedComputing/Project/"
file_distance = sc.textFile(path + "USA-BAY-distance.txt")
file_time = sc.textFile(path + "USA-BAY-time.txt")
file_coordinates = sc.textFile(path + "USA-BAY-coordinates.txt")

# print file_distance.count(), file_time.count(), file_coordinates.count()

def clean_coordinates(x):
    x_split = x.split(' ')
    return int(x_split[1]), (float(x_split[3]) / 1000000, float(x_split[2]) / 1000000)    # Return lattitude, longitude

def clean_files(x, metric):
    x_split = x.split(' ')
    if metric == 'distance':
        return int(x_split[1]), (int(x_split[2]), int(x_split[3]) / 10.0)
    else:
        return int(x_split[1]), (int(x_split[2]), int(x_split[3]))
        
def conv(x, y):
    return x[0] + y[0], x[1] + y[1]

# Sort key and values
def sort_index(x):
    x = list(x)
    m = sorted(x[0])
    n = [x[1] for (x[0],x[1]) in sorted(zip(x[0],x[1]))]
    return m, n

def haversine(node1, node2):
    
    R = 6372800 # Earth radius in meters
    dLat = radians(coordinates[node2 - 1][1][0] - coordinates[node1 - 1][1][0])
    dLon = radians(coordinates[node2 - 1][1][1] - coordinates[node1 - 1][1][1])
    lat1 = radians(coordinates[node1 - 1][1][0])
    lat2 = radians(coordinates[node2 - 1][1][0])
    
    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))
    
    return R * c

def lowest_f_score(opens, fscores):
    scores = []
    for x in opens:
        scores.append((x, fscores[x]))
    n = sorted(scores, key=lambda x: x[1])
    return n[0][0]

def reconstruct_path(history, curr, start):
    goal = curr
    rev = lambda l: [l[-1]] + (rev(l[:-1]) if len(l)>1 else [])
    path = []
    path.append(curr)
    while (curr != start):
        curr = history[curr]
        path.append(curr)
        
    constructed_path = '-'.join(str(i) for i in rev(path))
    print "Shortest path from node %d to node %d is" %(start, goal), constructed_path

    
distance_data = file_distance.map(lambda x: clean_files(x, 'distance'))
coor_data = file_coordinates.map(lambda x: clean_coordinates(x))
coordinates = coor_data.collect()

sorted_dist = distance_data.map(lambda x: ((x[0]), ([x[1][0]], [x[1][1]]))).reduceByKey(lambda x, y: conv(x, y)).sortByKey(ascending = True).mapValues(lambda x: sort_index(x))
all_distances = sorted_dist.collect()

start = 1
goal = 4
closed_set = []
open_set = []
came_from = {}

g_score = {}
f_score = {}
g_score[start] = 0
f_score[start] = 0
# Initialize open_set with the start node
open_set.append(start)

q = 1
while(len(open_set) != 0):
    if (q % 10 == 0):
        print "Iteration: ", q
    
    current = lowest_f_score(open_set, f_score)
    if current == goal:
        open_set = []
        reconstruct_path(came_from, current, start)
    
    else:
        open_set.remove(current)
        closed_set.append(current)
        for i in xrange(0, len(all_distances[current - 1][1][0])):
            neighbor = all_distances[current - 1][1][0][i]
            if neighbor in closed_set:
                continue

            tentative_g_score = g_score[current] + all_distances[current - 1][1][1][i]  ## Calculate f_score

            if neighbor not in open_set:
                open_set.append(neighbor)

            elif tentative_g_score >= g_score[neighbor]:
                continue

            came_from[neighbor] = current
            g_score[neighbor] = tentative_g_score
            f_score[neighbor] = g_score[neighbor] + haversine(neighbor, goal)
    q += 1
    
print "\nMinimum distance required to travel between nodes %d and %d is %.4f km" % (start, goal, g_score[goal] / 1000)
print "\nTotal code execution time is %.4f sec" % (time.time() - s)

Shortest path from node 1 to node 4 is 1-308-309-3-4

Minimum distance required to travel between nodes 1 and 4 is 1.2376 km

Total code execution time is 23.6649 sec
